### Alphabet Soup Charity Optimization Model 3

In [ ]:
%pip install keras-tuner --upgrade
# Import our dependencies
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn import __SKLEARN_SETUP__
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.utils import np_utils
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
csv_path = 'Resources/charity_data.csv'


#  Import and read the charity_data.csv.
application_df = pd.read_csv(csv_path)
application_df.head()

In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'], axis=1)
application_df.head()


In [ ]:
# Determine the number of unique values in each column.
application_df.nunique()


In [ ]:
# Look at APPLICATION_TYPE value counts for binning
app_counts = application_df['APPLICATION_TYPE'].value_counts()
app_counts


In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
application_types_to_replace = list(app_counts[app_counts < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(
        app, "Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()


In [ ]:
# Look at CLASSIFICATION value counts for binning
class_counts = application_df['CLASSIFICATION'].value_counts()
class_counts


In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_counts_gt1 = class_counts.loc[class_counts > 1]
class_counts_gt1.head()


In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
classifications_to_replace = list(class_counts[class_counts < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(
        cls, "Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()


In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
application_numeric = pd.get_dummies(application_df)


In [ ]:
# Split our preprocessed data into our features and target arrays
X = application_numeric.drop(['IS_SUCCESSFUL'], axis=1)
y = application_numeric['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=58)


In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


## Compile, Train and Evaluate the Model

In [ ]:
# Create a method that creats a new Sequential model with hyperparameter options

def create_model(hp):
    nn_model3 = tf.keras.models.Sequential()

    # Allow kerastuner to devide which activation function to use in hidden layers
    activation = hp.Choice('activation', ['sigmoid', 'tanh'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model3.add(tf.keras.layers.Dense(units=hp.Int('first_units', 
        min_value=1,
        max_value=5,
        step=5), activation=activation, input_dim=43))
    
    # Allow kerastuner to decide number of hidden layers and neurons to add to hidden layers
    for i in range(hp.Int('num_layers', 1, 5)):
        nn_model3.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=5,
            step=5),
            activation=activation))
        
    nn_model3.add(tf.keras.layers.Dense(units=1, activation='relu'))

    # Compile the model
    nn_model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return nn_model3

# Create kerastuner
import keras_tuner as kt

tuner = kt.Hyperband(create_model,
                     objective='val_accuracy',
                     max_epochs=5,
                     hyperband_iterations=2)


# Search for the best hyperparameters
tuner.search(X_train_scaled, y_train, epochs=5, validation_data=(X_test_scaled, y_test))

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=3)[0]

# Build the model with the best hyperparameters
model_3 = create_model(best_hps)

# Train the model
fit_model = model_3.fit(X_train_scaled, y_train, epochs=5)


In [ ]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective='val_accuracy',
    max_epochs=5,
    hyperband_iterations=2)



In [ ]:
# Run the kerastuner search for the best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled, y_test))

In [ ]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
for param in best_hyper:
    print(param.values)


In [ ]:
# Evaluate the best model against full test data
best_model = tuner.get_best_models(3)
for model in best_model:
    model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(F'Loss: {model_loss}, Accuracy: {model_accuracy}')

In [ ]:
# Export our model to HDF5 file
# Define the filename
filename = 'H5_Files/AlphabetSoupCharity_Optimization3.h5'


# Save the model to a HDF5 file
best_model.save(filename)
